In [68]:
import binascii
import sys
import traceback
from block import Block
from helper import read_varint
from tx import Tx


In [66]:
def hexToInt(value):
    return int(binascii.hexlify(value), 16)

with open("blocks_demo/blk00000.dat","rb") as block_file:
    for block in range(10):
        magicNumber = binascii.hexlify(block_file.read(4))
        blockSize = int.from_bytes(block_file.read(4),"little")
        version = int.from_bytes(block_file.read(4),"little")
        previousHash = binascii.hexlify(block_file.read(32))
        merkleHash = binascii.hexlify(block_file.read(32))
        creationTimeTimestamp = int.from_bytes(block_file.read(4),"little")
        #creationTime = datetime.datetime.fromtimestamp(creationTimeTimestamp).strftime('%d.%m.%Y %H:%M')
        bits = int.from_bytes(block_file.read(4),"little")
        nonce = int.from_bytes(block_file.read(4),"little")
        countOfTransactions = read_varint(block_file)
        print(magicNumber)
        print(blockSize)
        print(version)
        print(previousHash)
        print(merkleHash)
        print(bits)
        print(nonce)
        print(countOfTransactions)
        for transaction in range(countOfTransactions):
            tx = Tx.parse(block_file)
        
    
    
    

b'f9beb4d9'
285
1
b'0000000000000000000000000000000000000000000000000000000000000000'
b'3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a'
486604799
2083236893
1
b'01000000'
1
0
b'00000000000000000000000000000000000000000000000000ffffffff4d04ff'
b'ff001d0104455468652054696d65732030332f4a616e2f32303039204368616e'
1864397423
1919033454
105
b'6e6b206f'
1702043750
1684959075
b'206261696c6f757420666f722062616e6b73ffffffff0100f2052a0100000043'
b'4104678afdb0fe5548271967f1a67130b7105cd6a828e03909a67962e0ea1f61'
4014751676
1442038840
4
b'e51ec112'
1295539422
2366356215
b'578a4c702b6bf11d5fac00000000f9beb4d9d7000000010000006fe28c0ab6f1'
b'b372c1a6a246ae63f74f931e8365e15a089c68d6190000000000982051fd1e4b'
3995012712
2709219092
163
b'c3540bf7'
112643505
1042503656
b'0e61bc6649ffff001d01e3629901010000000100000000000000000000000000'
b'00000000000000000000000000000000000000ffffffff0704ffff001d0104ff'
705032704
1
67
b'410496b5'
1364453432
745173660
b'91e61ec11600ae1390813a627c66fb8be794

In [47]:
with open("blocks_demo/blk00000.dat","rb") as block_file:
    for line in range(10):
        charlist =[]
        for ch in range(100):
            char=""
            try:
                #print(binascii.hexlify(block_file.read(60)))
                char = block_file.read(1).decode("ascii")
                #print(type(char))
            except:
                char = binascii.hexlify(block_file.read(1)).decode("utf-8")
                #print(type(char))
                #print([( block_file.read(1).decode("ascii")). for x in range(60) ])
            charlist.append(char)
            

        print("".join(charlist))       
        #except Exception as e:
        #    excType, excValue, excTraceback = sys.exc_info()
        #    print(excType, excValue, excTraceback, file = sys.stdout)
    

bed9                                     ;ed7a{7a2c>gv611b88512:b84b^J)5fIff 2b|                        
           ffffMff EThe Times 03/Jan/2009 Chancellor on brink of second bailout for banksffff 05*   
CAgfdfeUH'ga6q010\a8(39	79beaab6Ibc?L38f3U1e125c8Mba574cp+k1d_00   bed900     o8c
f172a646634f1e6556819     20Q1eK44beh14g{a354b1b657
#>a66Iff 6201                                   ffffff ffff 05*   CAb5853Q72j,e616 1381:b|f8b94{3cR753715e00414
e6"72f62s2c23B58ac    bed900     H`181b 7e908aBuAo51Y86h9a00   fd54%1czZed48Xbbf\3674N2c1`"0fb066Iff bda                  
                 ffffff ffff 05*   CAr245bPR(c319Laa56df7b97a@@c0se690d4f8R721g3e#dF1779fc41*31kw00   bed900     ddcca3a1
081a]p0a7bb66bc_bj    D72"`d8]a9fb5f09877bb7b776|1c99]66Iff edm                                   ffffff ffff 05*
   CAb9e7l[)f995a4da7c60ad80fff19#89qa00c'&749ef9u3e5P9bo<aa00   bed900     IDF621c,ta550bo>@baf2a355